In [9]:
import os
from omop_etl.io import to_csv
from omop_etl.load import Loader
from omop_etl.utils import timeitc, find

rpath = '//share.***REMOVED***/Cancer_Center/data_release/v0'
config_path = '//share.***REMOVED***/Cancer_Center/omop_cc_etl/config.yml'
folders = ['clinical_data', 'health_system', 'vocabulary']

for folder in folders:
    dirpath = os.path.join(rpath, folder)
    
    try:
        os.mkdir(dirpath)
    except (FileExistsError) as e:
        print(f'Directory {folder} already exists. Nothing done.')
        pass

loader = Loader(config_path)
server = loader.config.server
database = loader.config.project_database

Directory clinical_data already exists. Nothing done.
Directory health_system already exists. Nothing done.
Directory vocabulary already exists. Nothing done.


In [2]:
hipaa = loader.list_tables(in_schema=['hipaa'])
xref = loader.list_tables(in_schema=['xref'])

In [3]:
# hipaa = hipaa[~find(['death','visit_occurrence','condition_occurrence','procedure_occurrence','drug_exposure'], hipaa.Table)]

In [4]:
# loader._proj_connection_str

In [5]:
xref = xref[~find(['mapping','cdm'], xref.Table)].copy()

In [6]:
xref['Rows'] = xref.Table.apply(lambda s: loader.row_count(s, schema='xref'))
xref

,Schema,Table,Rows
5,xref,CONCEPT,6581661
6,xref,VOCABULARY,92
7,xref,DOMAIN,48
8,xref,CONCEPT_CLASS,414
9,xref,CONCEPT_RELATIONSHIP,47733858
10,xref,RELATIONSHIP,618
11,xref,CONCEPT_SYNONYM,2180766
12,xref,CONCEPT_ANCESTOR,73470571
13,xref,SOURCE_TO_CONCEPT_MAP,167
14,xref,DRUG_STRENGTH,2829860


In [13]:
# with timeitc('Exporting csv files'):
#     dirpath = rpath
#     for table in hipaa.Table[3:]:
#         print('Exporting table:', table)
#         to_csv(dirpath, table, 2000, 'hipaa', server, database)
#     to_csv(dirpath, 'drug_exposure', 2000, 'hipaa', server, database)

In [7]:
xref.Table.to_list()

['CONCEPT',
 'VOCABULARY',
 'DOMAIN',
 'CONCEPT_CLASS',
 'CONCEPT_RELATIONSHIP',
 'RELATIONSHIP',
 'CONCEPT_SYNONYM',
 'CONCEPT_ANCESTOR',
 'SOURCE_TO_CONCEPT_MAP',
 'DRUG_STRENGTH']

In [1]:
# with timeitc('Exporting csv files'):
#     dirpath = os.path.join(rpath, 'vocabulary')
#     for table in ['CONCEPT', 'VOCABULARY', 'DOMAIN', 'CONCEPT_CLASS', 'RELATIONSHIP', 'CONCEPT_SYNONYM', 'SOURCE_TO_CONCEPT_MAP', 'DRUG_STRENGTH']:
#         to_csv(dirpath, table, 1000, 'xref', server, database)

In [6]:
# import os

# os.chdir('Z:\OMOP\omop_etl')
# print(os.getcwd())

# import pandas as pd
# from omop_etl.datastore import DataStore
# import pandas as pd
# from turbodbc import connect, make_options, Megabytes, Rows
# import time
# import math
# import numpy as np
# from pydma.databases import OneFLDb
# from pydma.utils import find

#from https://stackoverflow.com/questions/5478351/python-time-measure-function
# from contextlib import contextmanager

# @contextmanager
# def timeit_context(name=None):
#     startTime = time.time()
#     yield
#     elapsedTime = time.time() - startTime
#     print('{} finished in {}'.format(name, 
#                                      time.strftime("%H h %M m %S s", 
#                                      time.gmtime(elapsedTime))))

# table = "measurement"
# batch_size = 1_000_000
# omop = OneFLDb('edw', database='DWS_OMOP')
# tables = ['measurement']#'device_exposure', 'person', 'death', 'location', 'provider', 'care_site', 'visit_occurrence', 
# #           'drug_exposure','condition_occurrence', 'observation', 'procedure_occurrence', 'observation_period']
# rpath = '//share.***REMOVED***/COVID-19/OMOP/data_releases/v7'
# #omop.list_tables(in_schema=['hipaa'])
# # print(tables.Table.to_list())
# # n_batches = 40

# def nrows(table):
#     """Returns table number of rows.
#     """
#     con = connect(driver='{SQL Server}', server='edw.***REMOVED***.edu', 
#                   database='DWS_OMOP', trusted_connection='yes')
#     cursor= con.cursor()
#     con.autocommit = True
#     cursor.execute("EXEC sp_spaceused N'hipaa.{}';".format(table))
#     result = cursor.fetchall()[0]
#     con.close()
    
#     return result

# table = 'drug_exposure'
# # for table in tables: 
# results = nrows(table)
# kb_row = int(results[2].split(' ')[0])/int(results[1])
# # n_batches = math.ceil(rows/batch_size)
# # # batch_size = math.ceil(rows/n_batches)
# # print('Total rows: ', rows, '\nBatch size: ', batch_size, '\nBatches:', n_batches)
# kb_row*1e6/1000

In [ ]:
# table = 'visit_occurrence'
# rpath = '//share.***REMOVED***/COVID-19/OMOP/data_releases/v7'
# tables = ['drug_exposure', 'visit_occurrence', 'measurement']

# for table in tables:
#     # Set batch size in number of rows
#     options = make_options(read_buffer_size=Megabytes(2000), 
#                            prefer_unicode=True,
#                            use_async_io=True,
#                            limit_varchar_results_to_max=True)

#     con = connect(driver='{SQL Server}', server='edw.***REMOVED***.edu', database='DWS_OMOP', 
#                   trusted_connection='yes', turbodbc_options=options)

#     cursor = con.cursor()
#     cursor.execute(f"select * from hipaa.{table}")
#     batches = cursor.fetchnumpybatches()

#     count = 0
#     file = os.path.join(rpath, table + '.csv')

#     with timeit_context(f'Exporting {table}'):
#         if os.path.exists(file):
#             os.remove(file)

#         for batch in batches:

#             df = pd.DataFrame(batch)

#             if count == 0:
#                 dtypes = {t:batch[t].dtype.type for t in batch.keys()}
#                 for dtype in dtypes.keys():
#                     if dtypes[dtype] == np.int64:
#                         dtypes[dtype] = 'Int64'
#                     else:
#                         dtypes[dtype] = 'str'

#                 df = df.astype(dtypes)
#                 df.to_csv(file, index=False, sep='\t')

#             else: 
#                 df = df.astype(dtypes)
#                 df.to_csv(file, header=False, index=False, mode='a', sep='\t')

#             count =+ 1

#         con.close()

In [ ]:
# count = 0
# for batch in batches:
#     if count == 0:
#         df = pd.DataFrame(batch)
#         print('Rows:',df.shape)
#         print('Memory usage (MB):', df.memory_usage(deep=True).sum()/1e6)
#     else: break
#     count+=1

In [2]:
import pandas as pd
import math

In [10]:
# table = 'provider'
# file = os.path.join(rpath, table + '.csv')
# schema = 'hipaa'
# count = 1
# batch_size = 1000000

# with loader.engine.connect() as con:
#     result = con.execute("EXEC sp_spaceused N'hipaa.{}';".format(table))
#     rows = int(result.fetchall()[0][1].strip())

# n_batches = math.ceil(rows/batch_size)

# # print('Total rows: ', rows, '\nBatch size: ', batch_size, '\nBatches:', n_batches)

# with timeitc(f'Exporting {table}'):
#     if os.path.exists(file):
#         os.remove(file)
    
#     with loader.engine.connect() as con:
#         header = pd.read_sql(f'select top 0 * from {schema}.{table}', con)
#         header.to_csv(file, index=False, sep='\t')
#         for chunk in pd.read_sql(f'select * from {schema}.{table}', con, coerce_float=False, chunksize=batch_size):
#             chunk.to_csv(file, header=False, index=False, sep='\t', mode='a')
#             print(f'Batch {count}/{n_batches} complete.', end='\r')
#             count=+1

Exporting provider finished in 00:00:01
